In [1]:
import os
os.chdir('C:\\Users\\Admin\\Desktop\\')
import pandas as pd

df = pd.read_csv('Consumer_Complaints.csv')
df.info()

df=df[['Product','Consumer complaint narrative']]

df=df[df['Consumer complaint narrative'].notna()]

df.loc[df['Product'] == 'Credit reporting', 'Product'] = 'Credit reporting, credit repair services, or other personal consumer reports'
df.loc[df['Product'] == 'Credit card', 'Product'] = 'Credit card or prepaid card'
df.loc[df['Product'] == 'Payday loan', 'Product'] = 'Payday loan, title loan, or personal loan'
df.loc[df['Product'] == 'Virtual currency', 'Product'] = 'Money transfer, virtual currency, or money service'
df = df[df.Product != 'Other financial service']

def print_plot(index):
    example = df[df.index == index][['Consumer complaint narrative', 'Product']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Product:', example[1])
print_plot(10)

import re
import re #Regular Expression
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer #ed, ing etc
nltk.download('punkt')
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(clean_text)
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].str.replace('\d+', '')



from keras.preprocessing.text import Tokenizer
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Consumer complaint narrative'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))




from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(df['Consumer complaint narrative'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)


Y = pd.get_dummies(df['Product']).values
print('Shape of label tensor:', Y.shape)


from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.90, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)



from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import SpatialDropout1D
from keras.callbacks import ModelCheckpoint, EarlyStopping
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(13, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 2
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453883 entries, 0 to 1453882
Data columns (total 18 columns):
Date received                   1453883 non-null object
Product                         1453883 non-null object
Sub-product                     1218718 non-null object
Issue                           1453883 non-null object
Sub-issue                       901298 non-null object
Consumer complaint narrative    471691 non-null object
Company public response         538473 non-null object
Company                         1453883 non-null object
State                           1429850 non-null object
ZIP code                        1318996 non-null object
Tags                            198249 non-null object
Consumer consent provided?      845505 non-null object
Submitted via                   1453883 non-null object
Date sent to company            1453883 non-null object
Company response to consumer    1453882 non-null object
Timely response?                1453883 non-null obj

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Using TensorFlow backend.


Found 162911 unique tokens.
Shape of data tensor: (471399, 250)
Shape of label tensor: (471399, 13)
(47139, 250) (47139, 13)
(424260, 250) (424260, 13)


C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 42425 samples, validate on 4714 samples
Epoch 1/2
42425/42425 [==============================] - 379s 9ms/step - loss: 1.3491 - accuracy: 0.5718 - val_loss: 0.9952 - val_accuracy: 0.6996
Epoch 2/2
42425/42425 [==============================] - 512s 12ms/step - loss: 0.9907 - accuracy: 0.6985 - val_loss: 0.9120 - val_accuracy: 0.7251


In [4]:
df=df[['Product','Consumer complaint narrative']]

In [5]:
df

,Product,Consumer complaint narrative
0,Credit card or prepaid card,dissatisfied current outcome dispute initiated...
1,"Credit reporting, credit repair services, or o...",item showed credit report previously removed ...
2,Debt collection,saw credit report collection dont know since ...
3,"Credit reporting, credit repair services, or o...",capital one auto finacing unknown inquiring c...
4,Credit card or prepaid card,small trucking company one trucks needed repai...
...,...,...
471394,Vehicle loan or lease,letter sent eplaining final payment due since ...
471395,Debt collection,car loan regional acceptance language agreemen...
471396,Debt collection,eplanation provided contrary account handled s...
471397,Vehicle loan or lease,vehicle repossessed sale intent notification s...


In [3]:
import numpy as np
new_complaint = ['I am a victim of identity theft and someone stole my identity and personal information to open up a Visa credit card account with Bank of America. The following Bank of America Visa credit card account do not belong to me : XXXX.']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = ['Credit reporting, credit repair services, or other personal consumer reports', 'Debt collection', 'Mortgage', 'Credit card or prepaid card', 'Student loan', 'Bank account or service', 'Checking or savings account', 'Consumer Loan', 'Payday loan, title loan, or personal loan', 'Vehicle loan or lease', 'Money transfer, virtual currency, or money service', 'Money transfers', 'Prepaid card']
print(pred, labels[np.argmax(pred)])

[[1.16461795e-02 1.50541607e-02 2.36429665e-02 2.48980522e-01
  5.92438817e-01 3.63115370e-02 3.65626719e-03 4.51040105e-04
  1.66413747e-02 9.89105180e-03 1.27151108e-03 1.27218645e-02
  2.72927154e-02]] Student loan


In [9]:
import numpy as np
new_complaint = ['On XX/XX/XXXX, I received a letter from Harris and Zide representing XXXX XXXX. The original creditor was XXXX XXXX bank but sold to XXXX funding. The ']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = ['Credit reporting, credit repair services, or other personal consumer reports', 'Debt collection', 'Mortgage', 'Credit card or prepaid card', 'Student loan', 'Bank account or service', 'Checking or savings account', 'Consumer Loan', 'Payday loan, title loan, or personal loan', 'Vehicle loan or lease', 'Money transfer, virtual currency, or money service', 'Money transfers', 'Prepaid card']
print(pred, labels[np.argmax(pred)])

[[8.8476250e-03 6.5937825e-03 3.2309219e-02 4.9580202e-02 1.4086008e-01
  6.4284974e-01 2.4247628e-03 5.2719394e-04 3.4562804e-02 2.9739343e-02
  9.0429978e-04 2.3356322e-02 2.7444653e-02]] Bank account or service
